### Output : The person left (Exited) the bank or not

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [33]:
## Load the dataset
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [34]:
df.drop(columns=['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [35]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Encode Categorical Variables

In [36]:
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [37]:
df['Gender'].unique()

array([0, 1])

In [38]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

## One Hot Encode Geography column

In [39]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder(sparse_output=False)
geo_encoder=onehot_encoder_geo.fit_transform(df[['Geography']])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [40]:
geo_column_names=onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_column_names

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [41]:
geo_encoded_df=pd.DataFrame(data=geo_encoder,columns=geo_column_names)

In [42]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


## Combine the oneHotEncoded columns with original dataframe

In [43]:
df=pd.concat([df.drop(columns='Geography',axis=1),geo_encoded_df],axis=1)

In [44]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


## Save the Encoder and Scaler

In [45]:
with open("classificaiton_label_encoder_gender.pkl",'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open("classificaiton_onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo,file) 


## Divide the data into independent and dependent

In [46]:
X=df.drop("Exited",axis=1)
y=df['Exited']

In [47]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Standard Scaler

In [48]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [49]:
X_test

array([[-0.57749609,  0.91324755, -0.6557859 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.41876609,  0.91324755, -0.94100321, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.24540869,  0.91324755,  0.00972116, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [50]:
with open ("classificaiton_scaler.pkl",'wb') as file:
    pickle.dump(scaler,file)

### ANN Implementation

In [51]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


### Build Our ANN model

In [52]:
X_train.shape,X_train.shape[1]

((8000, 12), 12)

In [53]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1 Connected with input layer
    Dense(32,activation='relu'), # HL2
    Dense(1,activation="sigmoid") #output layer
])

In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Compile the Model

In [55]:
# opt = optimizers
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()

In [56]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

### Setup the TensorBoard

In [57]:
log_dir="logs/fit/"+datetime.datetime.now().strftime("%Y-%m-%d__%H-%M-%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
# histogram_freq=1 tells the TensorBoard callback to perform this action at the end of every single epoch.

### Setup Early Stopping

In [58]:
early_stopping_callback=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

### Training the Model

In [59]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3946 - accuracy: 0.8344 - val_loss: 0.3609 - val_accuracy: 0.8570
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3574 - accuracy: 0.8554 - val_loss: 0.3771 - val_accuracy: 0.8475
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3520 - accuracy: 0.8556 - val_loss: 0.3431 - val_accuracy: 0.8565
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3462 - accuracy: 0.8584 - val_loss: 0.3453 - val_accuracy: 0.8610
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3437 - accuracy: 0.8606 - val_loss: 0.3506 - val_accuracy: 0.8580
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3402 - accuracy: 0.8594 - val_loss: 0.3459 - val_accuracy: 0.8565
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3377 - accuracy: 0.8626 - val_loss: 0.3547 - val_accuracy: 0.8590

In [60]:
model.save("classification_model.h5")

c:\PYTHON\Python_Udemy\Projects\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Load TensorBoard Extension

In [61]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [62]:
%tensorboard --logdir logs/fit/

Reusing TensorBoard on port 6006 (pid 3412), started 1 day, 11:06:25 ago. (Use '!kill 3412' to kill it.)

### Load the Pickel File